In [2]:
import os
import sys
import numpy as np
import pandas as pd
import csv as csv
import tqdm.auto as tqdm
from multiprocessing import Pool
from importlib import reload  
import support as sup
reload(sup)
sys.path.insert(0, '../icenumerics')

import icenumerics as ice
from icenumerics.geometry import ordering 
import magcolloids as mgc

import matplotlib.pyplot as plt
import time
ureg = ice.ureg

In [3]:
output = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-1.2"

# Cairo Lattice 

    In this script I will run simulations in parallel with the following parameters.

In [4]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    # <To change the seed of the thermal noise>
    
    np.random.seed()
    
    # <Introduce the parameters for the simulation>

    sp = sup.spins()
    ureg = ice.ureg
    sp.create_lattice(geometry = "cairo",size =[l,l],lattice_constant=19.5458*ureg.um, border="closed spin")
    sp.order_spins(ordering.random_ordering)

    particle = ice.particle(radius = 5*ureg.um,
                 susceptibility = 0.005,
                 diffusion = 0.125*ureg.um**2/ureg.s,
                 temperature = 300*ureg.K,
                 density = 1000*ureg.kg/ureg.m**3)

    trap_long = ice.trap(trap_sep = 10*ureg.um,
                   height = 0.3*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    trap_short = ice.trap(trap_sep = 4.5258*ureg.um,
                   height = 0.3*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    traps = []

    # <In this loop, I will define which traps have to be shortere>

    for s in sp:

        if ( s.direction[0].magnitude == 1.0 or s.direction[0].magnitude == -1.0 or 
            s.direction[1].magnitude == 1.0 or s.direction[1].magnitude == -1.0 ):

            traps.append(trap_short)

        else:

            traps.append(trap_long) 

    col = ice.colloidal_ice(sp, particle, traps, height_spread = 0, susceptibility_spread = 0.1)

    col.pad_region(30*ureg.um) # Why is this important?? Whitout this line the simulation do not run.  
    
    # <Introduce the simulation parameters>
    
    world = ice.world(
        field = 50*ureg.mT,
        temperature = 300*ureg.K,
        dipole_cutoff = 200*ureg.um)
    
    total_time = 1500*ureg.s
    col.simulate(world,
                 name = "ACI_CairoLattice_50mT-1500s_l%u_exp%u"%(l,e),
                 include_timestamp = False,
                 targetdir = output,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])

    col.sim.field.fieldz = "v_Bmag*time/%f"%total_time.to(ureg.us).magnitude
    col.run_simulation()    
    
    name = os.path.split(col.sim.base_name)[1]
    with open(os.path.join(output,"index_Cairo_50mT-1500s.dat"),'a',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow([name, l, e])

In [5]:
e =  np.arange(10,50)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,10,10
1,11,10
2,12,10
3,13,10
4,14,10
5,15,10
6,16,10
7,17,10
8,18,10
9,19,10


In [6]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 10
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(output):
        os.makedirs(output)
    with open(os.path.join(output,"index_Cairo_50mT-1500s.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "l", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX


/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:63: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,1] = centers_toAddY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity i

scanning file


scanning file


scanning file


...

scanning file


scanning file


scanning file


scanning file


scanning file
